In [2]:
import os
import sys
import re
import numpy as np
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import select_nodes, to_param_dict, to_param_list, insert_zero_params, rescale_manybody_params
from statmechlib.preprocessing import Trajectory, scale_configuration, pair_dist_cutoff, find_index
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box

In [5]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [6]:
# stats_file = 'stats_marinica' # 'stats_samples'
# target_file = 'target_marinica' # 'target_samples'
params_file = 'marinica_params'
# stats_file = 'stats_manyknots' # 'stats_samples'
# target_file = 'target_manyknots' # 'target_samples'
# stats_file = 'stats_manymd' # 'stats_samples'
# target_file = 'target_manymd' # 'target_samples'
stats_file = 'stats_manyall' # 'stats_samples'
target_file = 'target_manyall' # 'target_samples'

In [7]:
with open(os.path.join(working, stats_file+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, target_file+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)

with open(os.path.join(working, params_file + '.pickle'), 'rb') as fi:
    param_list = pickle.load(fi)  

In [8]:
def downselect_knots(stats_inp, pair_knots, edens_knots):
    
    # find idices of knots
    pair_index = find_index(pair_knots, stats_inp['hyperparams']['pair'])
    edens_index = find_index(edens_knots, stats_inp['hyperparams']['edens'])
    
    # create boolean arrays with select indices set to True
    p_ix = np.array([True if i in pair_index else False for i in range(len(stats_inp['hyperparams']['pair']))])
    m_ix = np.array([True if i in edens_index else False for i in range(len(stats_inp['hyperparams']['edens']))])

    stats_out = select_nodes(stats_inp, p_ix, m_ix)
    
    return stats_out

In [32]:
pair_knots = [2.22, 2.46, 2.5648975,  2.629795 ,  2.6946925,  2.8663175,  2.973045 ,
        3.0797725,  3.5164725,  3.846445 ,  4.1764175,  4.700845 ,
        4.8953   ,  5.089755 ,  5.3429525,  5.401695 ,  5.4604375]
edens_knots = [ 2.5, 2.8663175, 3.1,  3.5,  4.9, 5.4604375]

In [33]:
stats = downselect_knots(stats_all, pair_knots, edens_knots)
print(stats['hyperparams'])

{u'pair': [2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.5, 2.8663175000000001, 3.1000000000000001, 3.5, 4.9000000000000004, 5.4604375000000003]}


In [34]:
for key in sorted(targets.keys()):
    for box, xyz, ene in zip(targets[key]['box'], targets[key]['xyz'], stats[key]['energy']):
        n = len(xyz)
        vol = np.linalg.det(box)
        corr = float(n*n)/vol
        ene.append([corr])

In [35]:
targets['eos_fcc']['weight'] = 0.0
targets['bain']['weight'] = 0.0
targets['eos']['weight'] = 0.0
targets['eos_bcc']['weight'] = 0.0
#targets['liq_5000K']['weight'] = 0.1

In [36]:
targets.keys()

['bcc_npt_langevin_3700K',
 'md',
 'vac_npt_langevin_2000K',
 'relax',
 'screw_111_npt_langevin_2000K',
 'bcc_npt_langevin_300K',
 'i111_npt_langevin_2000K',
 'eos',
 'fcc_npt_langevin_300K',
 'eos_fcc',
 'i110_npt_langevin_2000K',
 'liq_3700K',
 'liq_5000K',
 'eos_bcc',
 'bain']

In [37]:
[(key, targets[key]['weight']) for key in targets.keys()]

[('bcc_npt_langevin_3700K', 1.0),
 ('md', 1.0),
 ('vac_npt_langevin_2000K', 1.0),
 ('relax', 1.0),
 ('screw_111_npt_langevin_2000K', 1.0),
 ('bcc_npt_langevin_300K', 1.0),
 ('i111_npt_langevin_2000K', 1.0),
 ('eos', 0.0),
 ('fcc_npt_langevin_300K', 1.0),
 ('eos_fcc', 0.0),
 ('i110_npt_langevin_2000K', 1.0),
 ('liq_3700K', 1.0),
 ('liq_5000K', 1.0),
 ('eos_bcc', 0.0),
 ('bain', 0.0)]

In [38]:
start_params = []
# start_params.append([ -4.18353755e+00,  -9.63668936e-03,  0.0, -1.00171035e+02,
#          1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
#          2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
#          1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
#          1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
#         -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
#          4.64461212e+00,   3.23329113e-01,   5.82061842e-02, -0.1])
# start_params.append([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
#         -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
#         -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
#          2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
#         -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
#          2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
#         -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
#          5.51658454e-02,   1.58601447e+00])
# start_params.append([ -4.16739003e+00,  -4.97036785e-03,  -4.62426912e+02,
#          2.63469650e+02,   1.00253629e+01,   2.05151925e+01,
#         -1.36702712e+01,   2.29129977e+01,  -1.27210473e+01,
#          2.95533783e+00,   8.18470873e-01,  -7.89494754e-01,
#         -8.74455389e-01,   2.10177842e+00,  -9.22682795e-01,
#         -6.93852493e-02,  -1.52166575e+00,   1.40766925e+00,
#          1.52571934e+02,   3.37536873e-01,   6.89310740e-01,
#          4.87102227e-02,   2.79573955e+00])
# start_params.append([ -4.33954858e+00,  -1.02215197e-02,  0.0, -1.12590032e+02,
#          3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
#          2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
#          1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
#          1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
#         -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
#          4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
#          3.46319317e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02,   5.60287637e+00])

# start_params.append([-4.48347382e+00,  -7.78079006e-03,   2.92055760e+02,
#          1.84809430e+02,  -6.18705307e+01,   3.27735461e+01,
#         -1.36588262e+01,   2.11547368e+01,  -5.45394162e+00,
#          8.02998352e-01,   1.21949599e+00,  -9.43034331e-01,
#         -7.71285316e-01,   2.00564540e+00,  -8.93039729e-01,
#         -9.92574423e-03,  -1.55106037e+00,   1.39898785e+00,
#          2.54912062e+02,   3.19484582e+00,  -3.49947883e-01,
#          6.32970386e-02,   4.13612869e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02])
# start_params.append([ -4.18353755e+00,  -9.63668936e-03, 0.0, 0.0, -1.00171035e+02,
#          1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
#          2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
#          1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
#          1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
#         -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
#          4.64461212e+00,   3.23329113e-01,   5.82061842e-02])

# start_params.append([ -4.25063877e+00,  -8.42423097e-03,   0.0, 1.97184032e+02,
#         -3.94829480e+01,   4.06593664e+01,   7.33612852e+00,
#         -1.09581336e+01,   1.95459483e+01,  -4.25340373e+00,
#          1.25504213e+00,   1.21896374e+00,  -8.92117958e-01,
#         -8.23211840e-01,   1.97233653e+00,  -8.75779516e-01,
#          1.58480925e-02,  -1.54604172e+00,   1.39139443e+00,
#          1.05631765e+02,   3.83941299e+00,  -8.75153019e-02,
#          6.40078984e-02])

# start_params.append([ -3.91475071e+00,  -3.18833461e-03,  -7.37124021e+02,
#          2.23383357e+02,   9.01948241e+01,  -3.51596924e+01,
#          3.15714136e+01,  -1.39034112e+01,   2.07549574e+01,
#         -5.72446724e+00,   8.88308215e-01,   1.25569595e+00,
#         -9.94558188e-01,  -7.00015224e-01,   1.98302690e+00,
#         -9.18391221e-01,  -4.22084319e-02,  -1.56102695e+00,
#          1.43988518e+00,   2.27701705e+02,   3.80702595e+00,
#         -3.47117647e-01,   7.63947436e-02])

# start_params.append([ -3.83941975e+00,  -3.25610049e-03,  -6.98402475e+02,
#          2.57119905e+02,   1.34752272e+02,  -4.28139668e+01,
#          3.00399954e+01,  -1.29550960e+01,   1.98802247e+01,
#         -5.48953801e+00,   8.51960615e-01,   1.25718763e+00,
#         -1.00036821e+00,  -6.95536165e-01,   1.98898864e+00,
#         -9.15368120e-01,  -5.17527170e-02,  -1.56672725e+00,
#          1.44900080e+00,   2.98444914e+02,   3.84162304e+00,
#         -3.67028046e-01,   8.04355711e-02, 0.0])

start_params.append([ -3.81479610e+00,  -2.43304068e-03,  -8.12805411e+02,
         2.16931015e+02,   5.03869280e+01,  -1.66232406e+01,
         2.63908505e+01,  -1.23282440e+01,   1.87341912e+01,
        -5.03303276e+00,   1.03688215e+00,   1.29073562e+00,
        -1.01117984e+00,  -6.98740481e-01,   1.99047934e+00,
        -9.21553018e-01,  -6.59553220e-02,  -1.56838578e+00,
         1.46161003e+00,   2.08067464e+02,   0.0, 3.78487826e+00,
        -1.76783006e-01,   7.33887913e-02,   4.90869584e-13])

multi_pars = start_params

In [39]:
def optimize_EAM_mp(pars, targets, stats, utot_func):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, None, 0.05, 1), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)

    return tuple([output[1], output[0]])

In [ ]:
params_output = []

pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, ftot_func=None, dl=0.05, verbose=1)

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, utot_func=utot_EAM_per_atom)

    # initial ordering
    print('Initial params:', multi_pars[-1])
    
    #distances = pool.map(get_sd, multi_pars)
    
    distances = [get_sd(multi_pars[-1])]
    
    optimal_params = zip(distances, multi_pars[-1:])
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(50):
        #optimal_params = pool.map(get_opt, m_pars)
        optimal_params = [get_opt(m_pars)]
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print("Iteration {}, best params: {}".format(i, best_params))

    print('Final Best params:', best_params)
    
    params_output.append(best_params)

pool.close()
pool.join()

('Initial params:', [-3.8147961, -0.00243304068, -812.805411, 216.931015, 50.386928, -16.6232406, 26.3908505, -12.328244, 18.7341912, -5.03303276, 1.03688215, 1.29073562, -1.01117984, -0.698740481, 1.99047934, -0.921553018, -0.065955322, -1.56838578, 1.46161003, 208.067464, 0.0, 3.78487826, -0.176783006, 0.0733887913, 4.90869584e-13])
loss 7.64618453564e-07 7.64618453564e-07 0.0
('Best params:', [(7.6461845356352464e-07, [-3.8147961, -0.00243304068, -812.805411, 216.931015, 50.386928, -16.6232406, 26.3908505, -12.328244, 18.7341912, -5.03303276, 1.03688215, 1.29073562, -1.01117984, -0.698740481, 1.99047934, -0.921553018, -0.065955322, -1.56838578, 1.46161003, 208.067464, 0.0, 3.78487826, -0.176783006, 0.0733887913, 4.90869584e-13])])
('ieam:', 0)
loss 7.64618453564e-07 7.64618453564e-07 0.0
loss 0.000867069399311 0.000867069399311 0.0
loss 1.56671764642e-06 1.56671764642e-06 0.0
loss 7.64778347222e-07 7.64778347222e-07 0.0
loss 9.00636519276e-07 9.00636519276e-07 0.0
loss 9.38627282952

loss 3.8813708444e-06 3.8813708444e-06 0.0
loss 1.315898729e-06 1.315898729e-06 0.0
loss 3.04291734328e-06 3.04291734328e-06 0.0
loss 1.3422487308e-06 1.3422487308e-06 0.0
loss 3.19389755548e-06 3.19389755548e-06 0.0
loss 1.3150737052e-06 1.3150737052e-06 0.0
loss 2.98601045462e-06 2.98601045462e-06 0.0
loss 1.31669709639e-06 1.31669709639e-06 0.0
loss 3.17563898704e-06 3.17563898704e-06 0.0
loss 1.2900332719e-06 1.2900332719e-06 0.0
loss 2.9141129491e-06 2.9141129491e-06 0.0
loss 1.10424697179e-06 1.10424697179e-06 0.0
loss 2.88575043894e-06 2.88575043894e-06 0.0
loss 1.08300175585e-06 1.08300175585e-06 0.0
loss 2.22478050166e-06 2.22478050166e-06 0.0
loss 1.13004537855e-06 1.13004537855e-06 0.0
loss 1.8128405322e-06 1.8128405322e-06 0.0
loss 2.31677267567e-06 2.31677267567e-06 0.0
loss 1.06450982272e-06 1.06450982272e-06 0.0
loss 2.03187208574e-06 2.03187208574e-06 0.0
loss 1.00421432984e-06 1.00421432984e-06 0.0
loss 1.84266188614e-06 1.84266188614e-06 0.0
loss 9.73712730412e-07 9.7

loss 7.73960662015e-07 7.73960662015e-07 0.0
loss 7.6640096036e-07 7.6640096036e-07 0.0
loss 7.75120814373e-07 7.75120814373e-07 0.0
loss 7.66166388859e-07 7.66166388859e-07 0.0
loss 7.7044567459e-07 7.7044567459e-07 0.0
loss 7.70594400719e-07 7.70594400719e-07 0.0
loss 7.73589062887e-07 7.73589062887e-07 0.0
loss 7.66143142074e-07 7.66143142074e-07 0.0
loss 7.70777379473e-07 7.70777379473e-07 0.0
loss 7.66286187476e-07 7.66286187476e-07 0.0
loss 7.69520753831e-07 7.69520753831e-07 0.0
loss 7.70569730208e-07 7.70569730208e-07 0.0
loss 7.66032070328e-07 7.66032070328e-07 0.0
loss 7.70114390485e-07 7.70114390485e-07 0.0
loss 7.65943498844e-07 7.65943498844e-07 0.0
loss 7.7153663596e-07 7.7153663596e-07 0.0
loss 7.65783089406e-07 7.65783089406e-07 0.0
loss 7.70524806093e-07 7.70524806093e-07 0.0
loss 7.6586879274e-07 7.6586879274e-07 0.0
loss 7.68427610908e-07 7.68427610908e-07 0.0
loss 7.69343555936e-07 7.69343555936e-07 0.0
loss 7.65844918226e-07 7.65844918226e-07 0.0
loss 7.68887825326

loss 7.64637294935e-07 7.64637294935e-07 0.0
loss 7.64682034928e-07 7.64682034928e-07 0.0
loss 7.64635349162e-07 7.64635349162e-07 0.0
loss 7.64657367777e-07 7.64657367777e-07 0.0
loss 7.6467541e-07 7.6467541e-07 0.0
loss 7.6463403088e-07 7.6463403088e-07 0.0
loss 7.64661291063e-07 7.64661291063e-07 0.0
loss 7.64700801692e-07 7.64700801692e-07 0.0
loss 7.64630294315e-07 7.64630294315e-07 0.0
loss 7.64666541101e-07 7.64666541101e-07 0.0
loss 7.6467553681e-07 7.6467553681e-07 0.0
loss 7.64632627329e-07 7.64632627329e-07 0.0
loss 7.64664450543e-07 7.64664450543e-07 0.0
loss 7.64680933361e-07 7.64680933361e-07 0.0
loss 7.64629719443e-07 7.64629719443e-07 0.0
loss 7.64667548521e-07 7.64667548521e-07 0.0
loss 7.6463051236e-07 7.6463051236e-07 0.0
loss 7.64671704093e-07 7.64671704093e-07 0.0
loss 7.64628898759e-07 7.64628898759e-07 0.0
loss 7.64678709359e-07 7.64678709359e-07 0.0
loss 7.64626742715e-07 7.64626742715e-07 0.0
loss 7.64664046624e-07 7.64664046624e-07 0.0
loss 7.64627703954e-07 7

loss 7.64618779301e-07 7.64618779301e-07 0.0
loss 7.64619161664e-07 7.64619161664e-07 0.0
loss 7.64619594426e-07 7.64619594426e-07 0.0
loss 7.64618740667e-07 7.64618740667e-07 0.0
loss 7.64618716243e-07 7.64618716243e-07 0.0
loss 7.64619472971e-07 7.64619472971e-07 0.0
loss 7.64618729342e-07 7.64618729342e-07 0.0
loss 7.64619115477e-07 7.64619115477e-07 0.0
loss 7.64618743997e-07 7.64618743997e-07 0.0
loss 7.64619329973e-07 7.64619329973e-07 0.0
loss 7.64618724901e-07 7.64618724901e-07 0.0
loss 7.6461864319e-07 7.6461864319e-07 0.0
loss 7.64619105929e-07 7.64619105929e-07 0.0
loss 7.64618718462e-07 7.64618718462e-07 0.0
loss 7.64618603889e-07 7.64618603889e-07 0.0
loss 7.64619073956e-07 7.64619073956e-07 0.0
loss 7.64618705362e-07 7.64618705362e-07 0.0
loss 7.64618940948e-07 7.64618940948e-07 0.0
loss 7.64619580886e-07 7.64619580886e-07 0.0
loss 7.64618640301e-07 7.64618640301e-07 0.0
loss 7.64618654958e-07 7.64618654958e-07 0.0
loss 7.64618718683e-07 7.64618718683e-07 0.0
loss 7.64618

...

loss 7.61523788541e-07 7.61523788541e-07 0.0
loss 7.61523762562e-07 7.61523762562e-07 0.0
loss 7.61523789652e-07 7.61523789652e-07 0.0
loss 7.61523812744e-07 7.61523812744e-07 0.0
loss 7.6152375701e-07 7.6152375701e-07 0.0
loss 7.61523773664e-07 7.61523773664e-07 0.0
loss 7.61523722149e-07 7.61523722149e-07 0.0
loss 7.61523656868e-07 7.61523656868e-07 0.0
loss 7.61523738359e-07 7.61523738359e-07 0.0
loss 7.61523754124e-07 7.61523754124e-07 0.0
loss 7.61523734584e-07 7.61523734584e-07 0.0
loss 7.61523763006e-07 7.61523763006e-07 0.0
loss 7.61523714822e-07 7.61523714822e-07 0.0
loss 7.61523698835e-07 7.61523698835e-07 0.0
loss 7.61523776995e-07 7.61523776995e-07 0.0
loss 7.6152369062e-07 7.6152369062e-07 0.0
loss 0.516212693516 0.516212693516 0.0
loss 7.61523786321e-07 7.61523786321e-07 0.0
loss 7.61523690397e-07 7.61523690397e-07 0.0
loss 7.61523706608e-07 7.61523706608e-07 0.0
loss 7.61523735916e-07 7.61523735916e-07 0.0
loss 7.61523673744e-07 7.61523673744e-07 0.0
loss 7.61523669081e-

loss 7.61521887399e-07 7.61521887399e-07 0.0
loss 7.6152163649e-07 7.6152163649e-07 0.0
loss 7.61521871633e-07 7.61521871633e-07 0.0
loss 7.61521850317e-07 7.61521850317e-07 0.0
loss 7.61521805242e-07 7.61521805242e-07 0.0
loss 7.61521781039e-07 7.61521781039e-07 0.0
loss 7.615217051e-07 7.615217051e-07 0.0
loss 7.61521813678e-07 7.61521813678e-07 0.0
loss 7.61521829668e-07 7.61521829668e-07 0.0
loss 7.61521685782e-07 7.61521685782e-07 0.0
loss 7.61521703546e-07 7.61521703546e-07 0.0
loss 7.61521649367e-07 7.61521649367e-07 0.0
loss 7.61521583642e-07 7.61521583642e-07 0.0
loss 7.61521280774e-07 7.61521280774e-07 0.0
loss 7.61521600294e-07 7.61521600294e-07 0.0
loss 7.61521579202e-07 7.61521579202e-07 0.0
loss 7.61521609622e-07 7.61521609622e-07 0.0
loss 7.61521519471e-07 7.61521519471e-07 0.0
loss 7.61521484833e-07 7.61521484833e-07 0.0
loss 7.61521633158e-07 7.61521633158e-07 0.0
loss 7.61521507925e-07 7.61521507925e-07 0.0
loss 7.61521396903e-07 7.61521396903e-07 0.0
loss 7.615213755

loss 7.61513683763e-07 7.61513683763e-07 0.0
loss 7.61513540546e-07 7.61513540546e-07 0.0
loss 7.61513611602e-07 7.61513611602e-07 0.0
loss 7.61513413316e-07 7.61513413316e-07 0.0
loss 7.61513214364e-07 7.61513214364e-07 0.0
loss 7.61513140868e-07 7.61513140868e-07 0.0
loss 7.61512856206e-07 7.61512856206e-07 0.0
loss 7.61511849902e-07 7.61511849902e-07 0.0
loss 7.61512933037e-07 7.61512933037e-07 0.0
loss 7.61512672131e-07 7.61512672131e-07 0.0
loss 7.61512845101e-07 7.61512845101e-07 0.0
loss 7.61512566216e-07 7.61512566216e-07 0.0
loss 7.61512547344e-07 7.61512547344e-07 0.0
loss 7.61512828011e-07 7.61512828011e-07 0.0
loss 7.6151237748e-07 7.6151237748e-07 0.0
loss 7.61512245363e-07 7.61512245363e-07 0.0
loss 7.61512312199e-07 7.61512312199e-07 0.0
loss 7.61512069505e-07 7.61512069505e-07 0.0
loss 7.61512025097e-07 7.61512025097e-07 0.0
loss 7.61511937609e-07 7.61511937609e-07 0.0
loss 7.61512236925e-07 7.61512236925e-07 0.0
loss 7.61511894978e-07 7.61511894978e-07 0.0
loss 7.61511

loss 7.61500826291e-07 7.61500826291e-07 0.0
loss 7.61500491221e-07 7.61500491221e-07 0.0
loss 7.6150076434e-07 7.6150076434e-07 0.0
loss 7.61500737685e-07 7.61500737685e-07 0.0
loss 7.61500225435e-07 7.61500225435e-07 0.0
loss 7.61499774905e-07 7.61499774905e-07 0.0
loss 7.61501415593e-07 7.61501415593e-07 0.0
loss 7.6150036021e-07 7.6150036021e-07 0.0
loss 7.61500295816e-07 7.61500295816e-07 0.0
loss 7.61499977409e-07 7.61499977409e-07 0.0
loss 7.6150031359e-07 7.6150031359e-07 0.0
loss 7.61501239959e-07 7.61501239959e-07 0.0
loss 7.61500122404e-07 7.61500122404e-07 0.0
loss 7.61500008936e-07 7.61500008936e-07 0.0
loss 7.61500512754e-07 7.61500512754e-07 0.0
loss 7.61500096875e-07 7.61500096875e-07 0.0
loss 7.61500049353e-07 7.61500049353e-07 0.0
loss 7.61501091184e-07 7.61501091184e-07 0.0
loss 7.61499866603e-07 7.61499866603e-07 0.0
loss 7.61500391738e-07 7.61500391738e-07 0.0
loss 7.61499581723e-07 7.61499581723e-07 0.0
loss 7.61499125419e-07 7.61499125419e-07 0.0
loss 7.614997193

loss 7.61497552002e-07 7.61497552002e-07 0.0
loss 7.61497573322e-07 7.61497573322e-07 0.0
loss 7.61497462521e-07 7.61497462521e-07 0.0
loss 7.61497562887e-07 7.61497562887e-07 0.0
loss 7.614976819e-07 7.614976819e-07 0.0
loss 7.61497543788e-07 7.61497543788e-07 0.0
loss 7.61497435205e-07 7.61497435205e-07 0.0
loss 7.61497495598e-07 7.61497495598e-07 0.0
loss 7.61497620166e-07 7.61497620166e-07 0.0
loss 7.61497625058e-07 7.61497625058e-07 0.0
loss 7.6149741611e-07 7.6149741611e-07 0.0
loss 7.61497443419e-07 7.61497443419e-07 0.0
loss 7.61497399012e-07 7.61497399012e-07 0.0
loss 7.61497414552e-07 7.61497414552e-07 0.0
loss 7.61497429214e-07 7.61497429214e-07 0.0
loss 7.61497609954e-07 7.61497609954e-07 0.0
loss 7.61497485392e-07 7.61497485392e-07 0.0
loss 7.61497373037e-07 7.61497373037e-07 0.0
loss 7.61497390137e-07 7.61497390137e-07 0.0
loss 7.61497565775e-07 7.61497565775e-07 0.0
loss 7.61497430323e-07 7.61497430323e-07 0.0
loss 7.61497613291e-07 7.61497613291e-07 0.0
loss 7.614974802

In [60]:
params_output = [(8.9710060363567114e-07, np.array([ -4.18353755e+00,  -9.63668936e-03,  -1.00171035e+02,
         1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
         2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
         1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
         1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
        -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
         4.64461212e+00,   3.23329113e-01,   5.82061842e-02]))]

In [73]:
params_output = [(8.829397265807305e-07, np.array([ -4.33954858e+00,  -1.02215197e-02,  -1.12590032e+02,
         3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
         2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
         1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
         1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
        -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
         4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
         3.46319317e+00]))]

In [114]:
params_output = [(8.7421414571196454e-07, np.array([ -4.31249172e+00,  -9.93971047e-03,  -3.76018390e+01,
        -1.14759006e+02,   1.72557600e+01,   2.66437414e+01,
        -1.50817090e+01,   2.22342349e+01,  -3.62963119e+00,
         1.62230813e+00,   1.18480877e+00,  -8.52354746e-01,
        -8.38460594e-01,   1.98957701e+00,  -8.86622667e-01,
         1.66082225e-03,  -1.54824194e+00,   1.39761513e+00,
        -5.06767503e+01,   4.47029066e+00,   9.10205250e-02,
         5.87302830e-02,   1.82639066e+00]))]

In [19]:
params_output = [(8.8257346852300666e-07, np.array([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
        -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
        -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
         2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
        -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
         2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
        -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
         5.51658454e-02,   1.58601447e+00]))]

In [20]:
params_output = [(7.7799985353125938e-07, np.array([ -3.91475071e+00,  -3.18833461e-03,  -7.37124021e+02,
         2.23383357e+02,   9.01948241e+01,  -3.51596924e+01,
         3.15714136e+01,  -1.39034112e+01,   2.07549574e+01,
        -5.72446724e+00,   8.88308215e-01,   1.25569595e+00,
        -9.94558188e-01,  -7.00015224e-01,   1.98302690e+00,
        -9.18391221e-01,  -4.22084319e-02,  -1.56102695e+00,
         1.43988518e+00,   2.27701705e+02,   3.80702595e+00,
        -3.47117647e-01,   7.63947436e-02]))]

In [21]:
params_output = [(7.7598178840335949e-07, np.array([ -3.83941975e+00,  -3.25610049e-03,  -6.98402475e+02,
         2.57119905e+02,   1.34752272e+02,  -4.28139668e+01,
         3.00399954e+01,  -1.29550960e+01,   1.98802247e+01,
        -5.48953801e+00,   8.51960615e-01,   1.25718763e+00,
        -1.00036821e+00,  -6.95536165e-01,   1.98898864e+00,
        -9.15368120e-01,  -5.17527170e-02,  -1.56672725e+00,
         1.44900080e+00,   2.98444914e+02,   3.84162304e+00,
        -3.67028046e-01,   8.04355711e-02]))]

In [62]:
# convert lists of optimal parameters to dictionaries
params_optimal = []
for pars in params_output:
    #params_optimal.append(to_param_dict(pars, [2, 15, 4, 0]))
    params_optimal.append(to_param_dict(pars[1], stats['hyperparams']))

In [64]:
# store hyperparameters and output parameters in a single file
try:
    with open(os.path.join(working, 'params_store'+'.pickle'), 'rb') as fi:
        params_store = pickle.load(fi)    
except IOError:
    print('no file')
    params_store = []
    
params_store.extend(params_optimal)

with open(os.path.join(working, "params_store.pickle"), 'wb') as fo:
    pickle.dump(params_store, fo, protocol=2)

no file


In [65]:
with open(os.path.join(working, 'params_store' + '.pickle'), 'rb') as fi:
    retreived_params = pickle.load(fi)  

In [66]:
retreived_params

[{'hyperparams': {u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004],
   u'pair': [2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': array([-43.2896107 ,   4.64461212,   0.32332911,   0.05820618]),
   'embed': array([-4.18353755, -0.00963669]),
   'lrcorr': array([], dtype=float64),
   'pair': array([ -1.00171035e+02,   1.90025394e+01,   2.15938317e+01,
           -1.39759833e+01,   2.16332289e+01,  -3.29542126e+00,
            1.70455674e+00,   1.41347064e+00,  -9.02958785e-01,
           -8.62309098e-01,   1.95964588e+00,  -8.70527088e-01,
            3.22342700e-02,  -1.53866121e+00,   1.37095441e+00])}}]